In [1]:
import numpy as np
from scipy.special import factorial,multigammaln
from decimal import Decimal
import pandas as pd
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cluster

In [ ]:
## Define a new class called node which has the data points d_k and the number of the clusters


class Node:


    
    def __init__(self,p,alpha,i):
        
        self.single=True
        self.points=set()
        self.points.add(p)
        self.d=alpha
        self.number=i
        self.left=None
        self.right=None
        self.ph=0
## Add list that holds probability with other nodes
        self.pit={}
        self.rit={}


    def add(self,x):
        self.points.add(x)
        
    def add_all(self,x):
        self.points=x
        
    def remove(self,x):
        self.points.remove(x)
        
    def combine(self,y,alpha,i=None):
        ## Unless given a number set the number to the initial node
        p=self.points.union(y.points)
        z=Node(1,self.d,self.number)
        if i!=None:
            z.number=i
        z.left=self
        z.right=y
        z.remove(1)
        z.points=p
        z.d= alpha*factorial(len(p)-1)+self.d*y.d
        z.single=False
        return z 